In [2]:
%load_ext autoreload
%autoreload 2

%autosave 120
import sys
import pandas as pd
import glob
import numpy as np
import os
import random
from tqdm import tqdm
import json
from modules.utilities import are_bugs_from_tangled, has_bug, is_same_history, parse_hash_delimited_string
from modules.Untangler import Untangler

Autosaving every 120 seconds


In [5]:
import time


untangler = Untangler(model_name="openai")
untangler.change_model("gpt-4o")

csv_files = glob.glob("./data/Processed/*.csv")
for csv_file in tqdm(csv_files):
    df = pd.read_csv(csv_file,delimiter="\t")
    project_name = os.path.basename(csv_file).replace(".csv","")

    rows = []
    for _, row in tqdm(df.iterrows()):
        row = row.to_dict()
        history = parse_hash_delimited_string(row["Buggycommiit"])
        change_list = parse_hash_delimited_string(row["TangledWMoveandFileRename"])
        if row["Age"] < (365*2):
            continue
        if sum(history) >= 1 and any(t > 1 for t in [a * b for a, b in zip(change_list, history)]):
            row["Decision"] = "Buggy"
            change_list.reverse()
            history.reverse()
            
            json_path = os.path.join("./data/source-methods/", project_name, row["file"])
            data = json.load(open(json_path, encoding="utf8"))

            buggy = False

            for index, (change_count, is_buggy) in enumerate(zip(change_list, history)):
                if is_buggy == 1 and change_count > 1:
                    hash = data["changeHistory"][index]
                    
                    commit_data = data["changeHistoryDetails"][hash]
                    change_data = commit_data["subchanges"][0] if "subchanges" in commit_data.keys() else commit_data

                    #untangler.prepare_prompt(change_data["commitMessage"], change_data["diff"])

                    error = True
                    while error:
                        try:
                            result = "a"#untangler.detect()
                            error = False
                        except Exception as e:
                            error = True
                            print(f"Error - {e}.\nRetrying in 10s")
                            time.sleep(10)
                    if result == "Buggy":
                        buggy = True
                        break
                elif is_buggy == 1 and change_count == 1:
                    buggy = True
                    break
                    
            if buggy:
                row["Detection"] = "Buggy"
            else:
                row["Detection"] = "NotBuggy"
            row["result"] = result
        elif sum(history) >= 1:
            row["Decision"] = "Buggy"
            row["Detection"] = "Buggy"
        else:
            row["Decision"] = "NotBuggy"
            row["Detection"] = "NotBuggy"
        rows.append(row)
    df = pd.DataFrame(rows)
    df = df.drop(["Unnamed: 26"], axis=1)
    df.to_csv(f"./data/Cleaned/{project_name}.csv", index=False)

9781it [00:00, 11681.51it/s]<?, ?it/s]
12755it [00:01, 9471.60it/s]<00:51,  1.07s/it]
2659it [00:00, 4579.48it/s]2<01:10,  1.50s/it]
15953it [00:01, 11166.85it/s]00:52,  1.13s/it]
3340it [00:00, 7378.09it/s]5<01:03,  1.41s/it]
1145it [00:00, 11604.87it/s]<00:49,  1.12s/it]
2948it [00:00, 8457.38it/s]6<00:33,  1.27it/s]
1146it [00:00, 11993.50it/s]<00:28,  1.47it/s]
36514it [00:02, 13130.76it/s]00:20,  1.98it/s]
22124it [00:03, 6118.35it/s]<00:57,  1.43s/it]
60627it [00:05, 10540.60it/s]<01:29,  2.30s/it]
 20%|██        | 10/49 [00:20<01:21,  2.08s/it]


KeyboardInterrupt: 